In [ ]:
import time
import numpy as np
import pandas as pd
from collections import defaultdict
from IPython.display import clear_output, display

from rotman_lib import *

In [18]:
client = RITClient(api_key="ITCXWBRX")
resp = client.get_securities_book("RITC", limit=20)

In [ ]:
def stream_vwap_live_df_jupyter(client, tickers, poll_interval=0.2, limit=500):
    
    if isinstance(tickers, str):
        tickers = [tickers]

    last_id = {t: None for t in tickers}
    tick_pq = {t: defaultdict(float) for t in tickers}
    tick_q = {t: defaultdict(float) for t in tickers}

    df = pd.DataFrame(
        index=tickers,
        columns=["latest_tick", "vwap", "vol"],
    )

    clear_output(wait=True)
    display(df)

    while True:
        for tk in tickers:
            tas = client.get_securities_tas(tk, after=last_id[tk], limit=limit).json()
            if isinstance(tas, dict):
                rows = tas.get("data", tas.get("ticks", tas.get("tas", [])))
            else:
                rows = tas

            if rows:
                ids = []
                for r in rows:
                    rid = r.get("id")
                    if rid is not None:
                        ids.append(rid)

                    t = r.get("tick")
                    p = r.get("price")
                    q = r.get("quantity")
                    if t is None or p is None or q is None or q <= 0:
                        continue

                    tick_pq[tk][t] += p * q
                    tick_q[tk][t] += q

                if ids:
                    last_id[tk] = max(ids)

            if tick_q[tk]:
                latest_tick = max(tick_q[tk].keys())
                vwap = tick_pq[tk][latest_tick] / tick_q[tk][latest_tick]
                vol = tick_q[tk][latest_tick]
                df.loc[tk, "latest_tick"] = latest_tick
                df.loc[tk, "vwap"] = vwap
                df.loc[tk, "vol"] = vol
            else:
                df.loc[tk, "latest_tick"] = np.nan
                df.loc[tk, "vwap"] = np.nan
                df.loc[tk, "vol"] = 0

        clear_output(wait=True)
        display(df)

        time.sleep(poll_interval)


stream_vwap_live_df_jupyter(client, ["RITC", "COMP"], poll_interval=0.2)

,latest_tick,vwap,vol
RITC,599,10.53,68400.0
COMP,599,24.464704,77600.0


KeyboardInterrupt: 

In [ ]:
# display book depth and vwap

def _safe_float(x, default=np.nan):
    try:
        return float(x)
    except Exception:
        return default


def _sum_depth(levels, n_levels=5):
    """
    levels: list of dicts like [{"price":..., "quantity":...}, ...]
    returns (best_px, depth_sum)
    """
    if not levels:
        return (np.nan, 0.0)

    # Try common field names
    def get_px(lv):
        return lv.get("price", lv.get("px", lv.get("p")))

    def get_qty(lv):
        return lv.get("quantity", lv.get("qty", lv.get("size", lv.get("q"))))

    # Keep only valid levels
    cleaned = []
    for lv in levels:
        px = get_px(lv)
        qty = get_qty(lv)
        px = _safe_float(px)
        qty = _safe_float(qty, default=0.0)
        if np.isfinite(px) and qty > 0:
            cleaned.append((px, qty))

    if not cleaned:
        return (np.nan, 0.0)

    best_px = cleaned[0][0]
    depth_sum = sum(q for _, q in cleaned[:n_levels])
    return (best_px, depth_sum)


def stream_vwap_live_df_jupyter(
    client,
    tickers,
    poll_interval=0.2,
    limit=500,
    depth_levels=5,
    book_every=1,
):
    if isinstance(tickers, str):
        tickers = [tickers]

    last_id = {t: None for t in tickers}
    tick_pq = {t: defaultdict(float) for t in tickers}
    tick_q = {t: defaultdict(float) for t in tickers}

    df = pd.DataFrame(
        index=tickers,
        columns=[
            "latest_tick",
            "vwap",
            "vol",
            "bid_px",
            "ask_px",
            "spread",
            "bid_depth",
            "ask_depth",
            "imbalance",
        ],
        dtype=float,
    )

    loop_i = 0
    clear_output(wait=True)
    display(df)

    while True:

        case = client.get_case().json()
        tick = case.get("tick")
        period = case.get("period")
        status = case.get("status")
        if status in ("STOPPED", "ENDED", "FINISHED"):
            print(f"Streaming ended with status: {status}")
            break
        loop_i += 1

        for tk in tickers:
            # --- TAS / VWAP ---
            tas = client.get_securities_tas(tk, after=last_id[tk], limit=limit).json()
            if isinstance(tas, dict):
                rows = tas.get("data", tas.get("ticks", tas.get("tas", [])))
            else:
                rows = tas

            if rows:
                ids = []
                for r in rows:
                    rid = r.get("id")
                    if rid is not None:
                        ids.append(rid)

                    t = r.get("tick")
                    p = r.get("price")
                    q = r.get("quantity")
                    if t is None or p is None or q is None or q <= 0:
                        continue

                    tick_pq[tk][t] += p * q
                    tick_q[tk][t] += q

                if ids:
                    last_id[tk] = max(ids)

            if tick_q[tk]:
                latest_tick = max(tick_q[tk].keys())
                vwap = tick_pq[tk][latest_tick] / tick_q[tk][latest_tick]
                vol = tick_q[tk][latest_tick]
                df.loc[tk, "latest_tick"] = latest_tick
                df.loc[tk, "vwap"] = vwap
                df.loc[tk, "vol"] = vol
            else:
                df.loc[tk, "latest_tick"] = np.nan
                df.loc[tk, "vwap"] = np.nan
                df.loc[tk, "vol"] = 0.0

            # --- ORDER BOOK / DEPTH ---
            if loop_i % max(1, int(book_every)) == 0:
                try:

                    book = client.get_securities_book(tk).json()

                    if isinstance(book, dict):
                        payload = book.get("data", book.get("book", book))
                        bids = payload.get(
                            "bids", payload.get("buy", payload.get("bid", []))
                        )
                        asks = payload.get(
                            "asks", payload.get("sell", payload.get("ask", []))
                        )
                    else:
                        bids, asks = [], []

                    bid_px, bid_depth = _sum_depth(bids, n_levels=depth_levels)
                    ask_px, ask_depth = _sum_depth(asks, n_levels=depth_levels)

                    df.loc[tk, "bid_px"] = bid_px
                    df.loc[tk, "ask_px"] = ask_px
                    df.loc[tk, "bid_depth"] = bid_depth
                    df.loc[tk, "ask_depth"] = ask_depth

                    if np.isfinite(bid_px) and np.isfinite(ask_px):
                        df.loc[tk, "spread"] = ask_px - bid_px
                    else:
                        df.loc[tk, "spread"] = np.nan

                    denom = bid_depth + ask_depth
                    df.loc[tk, "imbalance"] = (
                        ((bid_depth - ask_depth) / denom) if denom > 0 else np.nan
                    )

                except Exception:
                    pass

        clear_output(wait=True)
        display(df)
        time.sleep(poll_interval)


pd.options.display.float_format = "{:.3f}".format

stream_vwap_live_df_jupyter(
    client,
    ["RITC", "COMP"],
    poll_interval=0.2,
    depth_levels=20,
    book_every=1,
)

,latest_tick,vwap,vol,bid_px,ask_px,spread,bid_depth,ask_depth,imbalance
RITC,598.000,9.419,90800.000,9.410,9.420,0.010,490200.000,601400.000,-0.102
COMP,599.000,25.440,32200.000,25.460,25.500,0.040,1439800.000,202800.000,0.753


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().